In [4]:
# %load classification/ts_classifier.py
from sklearn.metrics import classification_report
import matplotlib.pylab as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

class ts_classifier(object):
	
	def __init__(self,plotter=False):
		'''
		preds is a list of predictions that will be made.
		plotter indicates whether to plot each nearest neighbor as it is found.
		'''
		self.preds=[]
		self.plotter=plotter
	
	def predict(self,train,test,w,progress=False):
		'''
		1-nearest neighbor classification algorithm using LB_Keogh lower 
		bound as similarity measure. Option to use DTW distance instead
		but is much slower.
		'''
		for ind,i in enumerate(test):
			if progress:
				print str(ind+1)+' points classified'
			min_dist=float('inf')
			closest_seq=[]
	
			for j in train:
				if self.LB_Keogh(i,j[:-1],5)<min_dist:
					dist=self.DTWDistance(i,j[:-1],w)
					if dist<min_dist:
						min_dist=dist
						closest_seq=j
			self.preds.append(closest_seq[-1])
			
			if self.plotter: 
				plt.plot(i)
				plt.plot(closest_seq[:-1])
				plt.legend(['Test Series','Nearest Neighbor in Training Set'])
				plt.title('Nearest Neighbor in Training Set - Prediction ='+str(closest_seq[-1]))
				plt.show()
	    
	    
	def performance(self,true_results):
		'''
		If the actual test set labels are known, can determine classification
		accuracy.
		'''
		return classification_report(true_results,self.preds)
	
	def get_preds(self):
		return self.preds
	
	
	def DTWDistance(self,s1,s2,w=None):
		'''
		Calculates dynamic time warping Euclidean distance between two
		sequences. Option to enforce locality constraint for window w.
		'''
		DTW={}
    
		if w:
			w = max(w, abs(len(s1)-len(s2)))
    
			for i in range(-1,len(s1)):
				for j in range(-1,len(s2)):
					DTW[(i, j)] = float('inf')
			
		else:
		    for i in range(len(s1)):
		        DTW[(i, -1)] = float('inf')
		    for i in range(len(s2)):
		        DTW[(-1, i)] = float('inf')
		
		DTW[(-1, -1)] = 0
	
		for i in range(len(s1)):
			if w:
				for j in range(max(0, i-w), min(len(s2), i+w)):
					dist= (s1[i]-s2[j])**2
					DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
			else:
				for j in range(len(s2)):
					dist= (s1[i]-s2[j])**2
					DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
			
		return np.sqrt(DTW[len(s1)-1, len(s2)-1])
	   
	def LB_Keogh(self,s1,s2,r):
		'''
		Calculates LB_Keough lower bound to dynamic time warping. Linear
		complexity compared to quadratic complexity of dtw.
		'''
		LB_sum=0
		for ind,i in enumerate(s1):
	        
			lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
			upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
	        
			if i>upper_bound:
				LB_sum=LB_sum+(i-upper_bound)**2
			elif i<lower_bound:
				LB_sum=LB_sum+(i-lower_bound)**2
	    
		return np.sqrt(LB_sum)


In [24]:
ls

NAACL16-tsclust.ipynb                image2.pdf
README.md                            image3.pdf
acl16.ipynb                          imagetot.pdf
classification/                      saxpy.py
clustering/                          saxpy.pyc
datasets/                            time-series-Clustering-shuo15.ipynb
dendrogram.png                       toneData/
image1.pdf                           toneSub/


In [2]:
% matplotlib inline

In [6]:
clas=ts_classifier()


# non-OOP demo

In [64]:
from sklearn.metrics import classification_report
import matplotlib.pylab as plt
import numpy as np

def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return sqrt(DTW[len(s1)-1, len(s2)-1])


def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return sqrt(LB_sum)


from math import sqrt

def euclid_dist(t1,t2):
    return sqrt(sum((t1-t2)**2))


#see tutorial
from sklearn.metrics import classification_report

#knn with DTW using LB-Keogh
def knn(train,test,w):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    prec = precision_score(test[:,-1],preds, average='macro')
    recall= recall_score(test[:,-1],preds, average='macro')
    f1=f1_score(test[:,-1],preds, average='macro')

    return (prec,recall,f1)

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
#knn with Euclidean dist
def knn_euclid(train,test):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            dist=euclid_dist(i[:-1],j[:-1])
            if dist<min_dist:
                min_dist=dist
                closest_seq=j
        preds.append(closest_seq[-1])
    prec = precision_score(test[:,-1],preds, average='macro')
    recall= recall_score(test[:,-1],preds, average='macro')
    f1=f1_score(test[:,-1],preds, average='macro')

    return (prec,recall,f1)
#classification_report(test[:,-1],preds)

In [35]:
train = np.genfromtxt('datasets/train.csv', delimiter='\t')
test = np.genfromtxt('datasets/test.csv', delimiter='\t')
s = knn_euclid(train,test)


In [36]:
s

(0.89295320453420646, 0.87999999999999989, 0.86567390383305109)

In [32]:
type(p)

numpy.float64


# knn on our tone data - euclidean and dtw

In [9]:
import random

def split_data(data,test_size):
    test_ind = random.sample(range(len(data)),test_size)
    whole_ind = range(len(data))
    train_ind = [x for x in whole_ind if x not in test_ind]
    test = np.array([data[x] for x in test_ind])
    train = np.array([data[x] for x in train_ind])
    return test, train



from os import listdir
from os.path import isfile, join
mypath="toneData/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]




In [56]:
from os import listdir
from os.path import isfile, join
mypath="toneData/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]





num_iter=10

def addv(a,b):
    return tuple(map(sum, zip(a,b)))
    

#analysis:kmeans with euclidean distance with 5 types of representation feature files

score_dict={}
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    for i in range(num_iter):
        test,train=split_data(data,200)
        scores = knn_euclid(train,test)
        print "scores:",scores
        total_score=addv(total_score,scores)
        print "total score:",total_score
    ave_score=[x/num_iter for x in total_score]
    print ave_score
    score_dict[file]=ave_score
    print "======================================="

toneData/allxudn-3speakers-sorttone.csv
scores: (0.9278979954216906, 0.93482122715993687, 0.92970715055695297)
total score: (0.9278979954216906, 0.93482122715993687, 0.92970715055695297)
scores: (0.94390980776409106, 0.94393996960486326, 0.94378333600631481)
total score: (1.8718078031857817, 1.8787611967648001, 1.8734904865632678)
scores: (0.91754646002357676, 0.92192397751374888, 0.91911243618020022)
total score: (2.7893542632093586, 2.800685174278549, 2.7926029227434679)
scores: (0.95533626707638863, 0.95577321494635592, 0.95532303243878203)
total score: (3.7446905302857472, 3.756458389224905, 3.7479259551822501)
scores: (0.92969907407407404, 0.92966494555164148, 0.92896712812851001)
total score: (4.6743896043598214, 4.6861233347765463, 4.6768930833107598)
scores: (0.93599087433715633, 0.93936778065523907, 0.93369275078666081)
total score: (5.6103804786969782, 5.6254911154317853, 5.6105858340974208)
scores: (0.93876968020643969, 0.9382868937048503, 0.93828370046348619)
total score: (

In [61]:
score_dict['row-yixu-hertz-sorttone.csv']

[0.94936234291128352, 0.94930475041942231, 0.9487375513155818]

# SAX classificaiton knn (k=1 so far)


In [111]:
a = np.zeros(shape=(5,2))


In [112]:
a

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [116]:
a[3,1]=1.2

In [117]:
a

array([[ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  1.2],
       [ 0. ,  0. ]])

In [126]:
from saxpy import SAX
def min_dist_sax(t1String,t2String,word,alpha,eps=0.000001):
    s=SAX(word,alpha,eps)
    return s.compare_strings(t1String,t2String)


def convert_sax(ts,word,alpha,eps=0.000001):
    s=SAX(word,alpha,eps)
    (t1String, t1Indices) = s.to_letter_rep(ts)
    return t1String

In [148]:
a='sdgh'
a+'1'

'sdgh1'

In [153]:
#convert all data to sax
def data_sax(data,word,alpha):
    data_sax=[]
    for ts in data:
        ts_string=convert_sax(ts[:-1],word,alpha)
        ts_string+=str(int(ts[-1]))
        data_sax.append(ts_string)
    return data_sax


In [154]:
a=data_sax(data,word,alpha)

In [155]:
print a[:100]

['aabcddd1', 'aabcddc1', 'abccddd1', 'aabcddc1', 'aabbcdd1', 'aabcddd1', 'aacddcb1', 'aabcddd1', 'aacddcc1', 'aabcddd1', 'aaabcdd1', 'aabcddd1', 'aabccdd1', 'aabcddd1', 'aabcddd1', 'aabbcdd1', 'adddbba1', 'aabccdd1', 'acdddba1', 'aabbcdd1', 'aabccdd1', 'aacddcc1', 'bbcddba1', 'aabcddc1', 'aabcddd1', 'aabcddd1', 'baaacdd1', 'dbbcbaa1', 'aabcddd1', 'aabcddd1', 'dcbbabc1', 'aabcddd1', 'bbbcdda1', 'ccddbaa1', 'aabcddd1', 'aacdddc1', 'aabccdc1', 'aaabddd1', 'abccccd1', 'aabcddd1', 'aacddcc1', 'dbabdca1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'dddbaaa1', 'aaccddd1', 'aaabddd1', 'aabcddc1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'abbbcdd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'abddccc1', 'aabcddd1', 'abbbcdd1', 'aabcddd1', 'aabcddd1', 'aabbcdd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'acddcba1', 'bbddcaa1', 'abddcbb1', 'abbccdd1', 'aacddca1', 'dccdcaa1', 'abcddcc1', 'aabcddd1', 'aaacddd1', 'aacddcc1', 'abccbdd1', 'aa

In [189]:
data[0]

array([ -6.27248860e-02,  -5.98698290e-02,  -5.62488890e-02,
        -5.17472450e-02,  -4.83555100e-02,  -4.37595120e-02,
        -3.71938240e-02,  -2.98960330e-02,  -2.23343880e-02,
        -1.44712130e-02,  -7.18623600e-03,  -2.55626000e-04,
         6.09065600e-03,   1.12446530e-02,   1.56460630e-02,
         1.87748920e-02,   2.13038610e-02,   2.34514510e-02,
         2.51449490e-02,   2.68146640e-02,   2.84535490e-02,
         2.98095120e-02,   3.09127900e-02,   3.17744060e-02,
         3.17543630e-02,   3.12349470e-02,   3.01104630e-02,
         2.75978310e-02,   2.42377950e-02,   1.96863470e-02,
         1.00000000e+00])

In [157]:
len(all_sax)

10

In [158]:
print len(all_sax[2])

1920


In [167]:
min_dist_sax(all_sax[2][0][:-1],all_sax[2][988][:-1],5,3)

1.4895636945092345

In [183]:
def knn_sax(train,test,word,alpha):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #HeRE i and j are just two strings 'abcd1' and 'abcc2' e.g.
        for j in train:
            dist=min_dist_sax(i[:-1],j[:-1],word,alpha)
            if dist<min_dist:
                min_dist=dist
                closest_seq=j
                #print 'j',j
        preds.append(closest_seq[-1])
    #print 'preds',preds
    gtruth=[x[-1] for x in test]
    prec = precision_score(gtruth,preds, average='macro')
    recall= recall_score(gtruth,preds, average='macro')
    f1=f1_score(gtruth,preds, average='macro')

    return (prec,recall,f1)

In [217]:
#convert all data to sax
def data_sax(data,word,alpha):
    data_sax=[]
    for ts in data:
        ts_string=convert_sax(ts[:-1],word,alpha)
        ts_string+=str(int(ts[-1]))
        data_sax.append(ts_string)
    return data_sax

#try different parameter combinations for a data file (such as D1 data or bk data file)
def generate_sax_data(data):
    all_sax=[]
    all_pars=[]
    for word in range(5,):
        for alpha in range(6,7):
            a=data_sax(data,word,alpha)
            all_sax.append(a)
            all_pars.append((word,alpha))
    return all_sax,all_pars
        



In [214]:
from os import listdir
from os.path import isfile, join
mypath="toneSub/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]
all_data=[]
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    all_data.append(data)

toneSub/row-yixu-hertz-sorttone.csv


In [ ]:
#there are several data files, stored in all_data. for each data file in all data, you can generate multiple sax files
#by permutating the word and alpha values using the generate_sax_data() function. 


In [218]:
num_iter=1
score_dict_sax={}
for data_file in all_data:
    all_sax,all_pars=generate_sax_data(data_file)
    print 'generated file from ' + str(len(all_data)) + ' data file(s)'
    for i in range(len(all_sax)):
        data=all_sax[i]
        data_pars=all_pars[i]
        total_score=(0,0,0)
        print "sax parameters:",data_pars
        #fileName=mypath+file
        #print fileName
        #data = np.genfromtxt(fileName, delimiter=',')
        for i in range(num_iter):
            best_so_far=(0,0,0)
            test,train=split_data(data,200)
            scores = knn_sax(train,test,data_pars[0],data_pars[1])
            print 'scores:',scores
            if scores[-1]>best_so_far[-1]:
                best_so_far=scores                
            print "scores:",best_so_far
            total_score=addv(total_score,scores)
            print "total score:",total_score
        ave_score=[x/num_iter for x in total_score]
        print ave_score
        #score_dict_sax[file]=ave_score
        print "======================================="

generated file from 1 data file(s)
sax parameters: (3, 5)
scores: (0.057500000000000002, 0.25, 0.093495934959349603)
scores: (0.057500000000000002, 0.25, 0.093495934959349603)
total score: (0.057500000000000002, 0.25, 0.093495934959349603)
[0.057500000000000002, 0.25, 0.093495934959349603]
sax parameters: (3, 6)


KeyboardInterrupt: 

generated file from 1 data file(s)

sax parameters: (8, 3)

scores: (0.056250000000000001, 0.25, 0.091836734693877542)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
total score: (0.056250000000000001, 0.25, 0.091836734693877542)
[0.056250000000000001, 0.25, 0.091836734693877542]
 =======================================
sax parameters: (8, 4)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
total score: (0.056250000000000001, 0.25, 0.091836734693877542)
[0.056250000000000001, 0.25, 0.091836734693877542]
=======================================
sax parameters: (8, 5)
scores: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
scores: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
total score: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
[0.22758037225042299, 0.26111111111111113, 0.11879251700680271]
 =======================================
sax parameters: (8, 6)
scores: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
scores: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
total score: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
[0.26984588543181154, 0.31206948188080263, 0.22427140255009109]
=======================================
sax parameters: (9, 3)
scores: (0.068750000000000006, 0.25, 0.10784313725490198)
scores: (0.068750000000000006, 0.25, 0.10784313725490198)
total score: (0.068750000000000006, 0.25, 0.10784313725490198)
[0.068750000000000006, 0.25, 0.10784313725490198]
=======================================
sax parameters: (9, 4)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
total score: (0.071249999999999994, 0.25, 0.11089494163424124)
[0.071249999999999994, 0.25, 0.11089494163424124]
=======================================
sax parameters: (9, 5)
scores: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
scores: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
total score: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
[0.28423390081421168, 0.28061224489795916, 0.16288316946211681]
=======================================
sax parameters: (9, 6)
scores: (0.23060675883256529, 0.29499999999999998, 0.17955619747899157)
scores: (0.23060675883256529, 0.29499999999999998, 0.17955619747899157)
total score: (0.23060675883256529, 0.29499999999999998, 0.17955619747899157)
[0.23060675883256529, 0.29499999999999998, 0.17955619747899157]
=======================================
sax parameters: (10, 3)
scores: (0.059999999999999998, 0.25, 0.096774193548387094)
scores: (0.059999999999999998, 0.25, 0.096774193548387094)
total score: (0.059999999999999998, 0.25, 0.096774193548387094)
[0.059999999999999998, 0.25, 0.096774193548387094]
=======================================
sax parameters: (10, 4)
scores: (0.0625, 0.25, 0.10000000000000001)
scores: (0.0625, 0.25, 0.10000000000000001)
total score: (0.0625, 0.25, 0.10000000000000001)
[0.0625, 0.25, 0.10000000000000001]
=======================================
sax parameters: (10, 5)
scores: (0.25288018433179721, 0.30188679245283018, 0.1741948153967007)
scores: (0.25288018433179721, 0.30188679245283018, 0.1741948153967007)
total score: (0.25288018433179721, 0.30188679245283018, 0.1741948153967007)
[0.25288018433179721, 0.30188679245283018, 0.1741948153967007]
=======================================
sax parameters: (10, 6)
scores: (0.33011049723756908, 0.33962264150943394, 0.25328335657833567)
scores: (0.33011049723756908, 0.33962264150943394, 0.25328335657833567)
total score: (0.33011049723756908, 0.33962264150943394, 0.25328335657833567)
[0.33011049723756908, 0.33962264150943394, 0.25328335657833567]
=======================================
sax parameters: (11, 3)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
total score: (0.071249999999999994, 0.25, 0.11089494163424124)
[0.071249999999999994, 0.25, 0.11089494163424124]
=======================================
sax parameters: (11, 4)
scores: (0.061249999999999999, 0.25, 0.098393574297188743)
scores: (0.061249999999999999, 0.25, 0.098393574297188743)
total score: (0.061249999999999999, 0.25, 0.098393574297188743)
[0.061249999999999999, 0.25, 0.098393574297188743]
=======================================
sax parameters: (11, 5)
scores: (0.27994227994227994, 0.29999999999999999, 0.19621080139372821)
scores: (0.27994227994227994, 0.29999999999999999, 0.19621080139372821)
total score: (0.27994227994227994, 0.29999999999999999, 0.19621080139372821)
[0.27994227994227994, 0.29999999999999999, 0.19621080139372821]
=======================================
sax parameters: (11, 6)
scores: (0.29110135674381482, 0.36249999999999999, 0.26506235112792487)
scores: (0.29110135674381482, 0.36249999999999999, 0.26506235112792487)
total score: (0.29110135674381482, 0.36249999999999999, 0.26506235112792487)
[0.29110135674381482, 0.36249999999999999, 0.26506235112792487]
=======================================
sax parameters: (12, 3)
scores: (0.058749999999999997, 0.25, 0.095141700404858309)
scores: (0.058749999999999997, 0.25, 0.095141700404858309)
total score: (0.058749999999999997, 0.25, 0.095141700404858309)
[0.058749999999999997, 0.25, 0.095141700404858309]
=======================================

generated file from 1 data file(s)
sax parameters: (12, 6)
scores: (0.28259966777408635, 0.37522644927536231, 0.26898034398034398)
scores: (0.28259966777408635, 0.37522644927536231, 0.26898034398034398)
total score: (0.28259966777408635, 0.37522644927536231, 0.26898034398034398)
[0.28259966777408635, 0.37522644927536231, 0.26898034398034398]
=======================================
sax parameters: (12, 7)
scores: (0.53261904761904766, 0.3611111111111111, 0.26822188321044155)
scores: (0.53261904761904766, 0.3611111111111111, 0.26822188321044155)
total score: (0.53261904761904766, 0.3611111111111111, 0.26822188321044155)
[0.53261904761904766, 0.3611111111111111, 0.26822188321044155]
=======================================
sax parameters: (13, 6)
scores: (0.30225225225225227, 0.3214285714285714, 0.21742584745762711)
scores: (0.30225225225225227, 0.3214285714285714, 0.21742584745762711)
total score: (0.30225225225225227, 0.3214285714285714, 0.21742584745762711)
[0.30225225225225227, 0.3214285714285714, 0.21742584745762711]
=======================================
sax parameters: (13, 7)
scores: (0.28862126245847175, 0.37254901960784315, 0.26191448404596629)
scores: (0.28862126245847175, 0.37254901960784315, 0.26191448404596629)
total score: (0.28862126245847175, 0.37254901960784315, 0.26191448404596629)
[0.28862126245847175, 0.37254901960784315, 0.26191448404596629]
=======================================
sax parameters: (14, 6)
scores: (0.26486486486486488, 0.31122448979591838, 0.19675429184549356)
scores: (0.26486486486486488, 0.31122448979591838, 0.19675429184549356)
total score: (0.26486486486486488, 0.31122448979591838, 0.19675429184549356)
[0.26486486486486488, 0.31122448979591838, 0.19675429184549356]
=======================================
sax parameters: (14, 7)
scores: (0.26338742393509129, 0.34234084880636606, 0.25243664717348924)
scores: (0.26338742393509129, 0.34234084880636606, 0.25243664717348924)
total score: (0.26338742393509129, 0.34234084880636606, 0.25243664717348924)
[0.26338742393509129, 0.34234084880636606, 0.25243664717348924]
=======================================
sax parameters: (15, 6)
scores: (0.27604166666666663, 0.35106382978723405, 0.25285043594902745)
scores: (0.27604166666666663, 0.35106382978723405, 0.25285043594902745)
total score: (0.27604166666666663, 0.35106382978723405, 0.25285043594902745)
[0.27604166666666663, 0.35106382978723405, 0.25285043594902745]
=======================================

generated file from 1 data file(s)
sax parameters: (15, 6)
scores: (0.30440276669326949, 0.34134615384615385, 0.2492859224715826)
scores: (0.30440276669326949, 0.34134615384615385, 0.2492859224715826)
total score: (0.30440276669326949, 0.34134615384615385, 0.2492859224715826)
[0.30440276669326949, 0.34134615384615385, 0.2492859224715826]
=======================================
sax parameters: (15, 7)
scores: (0.28460038986354774, 0.39584135541008858, 0.28956825165355971)
scores: (0.28460038986354774, 0.39584135541008858, 0.28956825165355971)
total score: (0.28460038986354774, 0.39584135541008858, 0.28956825165355971)
[0.28460038986354774, 0.39584135541008858, 0.28956825165355971]
=======================================
sax parameters: (16, 6)
scores: (0.25353120824584846, 0.34318444358875627, 0.24240934906072523)
scores: (0.25353120824584846, 0.34318444358875627, 0.24240934906072523)
total score: (0.25353120824584846, 0.34318444358875627, 0.24240934906072523)
[0.25353120824584846, 0.34318444358875627, 0.24240934906072523]
=======================================
sax parameters: (16, 7)
scores: (0.29598662207357862, 0.4107142857142857, 0.2929219600725953)
scores: (0.29598662207357862, 0.4107142857142857, 0.2929219600725953)
total score: (0.29598662207357862, 0.4107142857142857, 0.2929219600725953)
[0.29598662207357862, 0.4107142857142857, 0.2929219600725953]
=======================================

# knn - (k>1)

In [239]:
# Example of kNN implemented from Scratch in Python

import csv
import random
import math
import operator

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
	with open(filename, 'rb') as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
	    for x in range(len(dataset)-1):
	        for y in range(4):
	            dataset[x][y] = float(dataset[x][y])
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])


def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
	
def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	split = 0.67
	loadDataset('iris.data', split, trainingSet, testSet)
	print 'Train set: ' + repr(len(trainingSet))
	print 'Test set: ' + repr(len(testSet))
	# generate predictions
	predictions=[]
	k = 3
	for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		#print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: ' + repr(accuracy) + '%')
	
main()

Train set: 102
Test set: 48
Accuracy: 97.91666666666666%


In [227]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b'],[3,3,3,'b'],[1,1,1,'a']]
testInstance = [5, 5, 5]
k = 3
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)
response = getResponse(neighbors)
print(response)

[[4, 4, 4, 'b'], [3, 3, 3, 'b'], [2, 2, 2, 'a']]
b


In [229]:
train[:10]

array(['aee1', 'ade1', 'aee1', 'acf1', 'adf1', 'aed1', 'adf1', 'aed1',
       'adf1', 'acf1'], 
      dtype='|S4')

In [238]:
from os import listdir
from os.path import isfile, join
mypath="toneData/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]





num_iter=4

def addv(a,b):
    return tuple(map(sum, zip(a,b)))
    

#analysis:kmeans with euclidean distance with 5 types of representation feature files
k=3
score_dict={}
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    for i in range(num_iter):
        predictions=[]
        test,train=split_data(data,200)
        for x in range(len(test)):
            neighbors = getNeighbors(train, test[x], k)
            result = getResponse(neighbors)
            predictions.append(result)
            #print('> predicted=' + repr(result) + ', actual=' + repr(test[x][-1]))
        accuracy = getAccuracy(test, predictions)
        print('Accuracy: ' + repr(accuracy) + '%')


toneData/allxudn-3speakers-sorttone.csv
Accuracy: 95.5%
Accuracy: 96.5%
Accuracy: 92.0%
Accuracy: 95.0%
toneData/concatSynth1-sorttone.csv
Accuracy: 84.0%
Accuracy: 81.5%
Accuracy: 79.0%
Accuracy: 75.0%
toneData/poly-normed-bark-30p-sorttone.csv
Accuracy: 76.5%
Accuracy: 81.5%
Accuracy: 75.5%
Accuracy: 77.0%
toneData/row-yixu-hertz-sorttone.csv
Accuracy: 97.0%
Accuracy: 96.0%
Accuracy: 93.5%
Accuracy: 96.0%
toneData/row-yixunorm-bk-sorttone.csv
Accuracy: 92.0%
Accuracy: 93.0%
Accuracy: 95.0%
Accuracy: 93.0%


# knn (k>1) SAX

In [241]:
from os import listdir
from os.path import isfile, join
mypath="toneSub/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]
all_data=[]
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    all_data.append(data)

toneSub/row-yixu-hertz-sorttone.csv


In [350]:
def data_sax(data,word,alpha):
    data_sax=[]
    for ts in data:
        ts_string=convert_sax(ts[:-1],word,alpha)
        ts_string+=str(int(ts[-1]))
        data_sax.append(ts_string)
    return data_sax

#try different parameter combinations for a data file (such as D1 data or bk data file)
def generate_sax_data(data):
    all_sax=[]
    all_pars=[]
    wordpars=[20]
    alphapars=[17,20]
    for word in wordpars:
        for alpha in alphapars:
            a=data_sax(data,word,alpha)
            all_sax.append(a)
            all_pars.append((word,alpha))
    return all_sax,all_pars

In [251]:
def getNeighborsSAX(trainingSet, testInstance, word,alpha):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = min_dist_sax(trainingSet[x][:-1],testInstance[:-1],word,alpha)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]


In [351]:

k=7

num_iter=1
score_dict_sax={}
for data_file in all_data:
    all_sax,all_pars=generate_sax_data(data_file)
    print 'generated file from ' + str(len(all_data)) + ' data file(s)'
    for i in range(len(all_sax)):
        data=all_sax[i]
        data_pars=all_pars[i]
        total_score=(0,0,0)
        print "sax parameters:",data_pars
        #fileName=mypath+file
        #print fileName
        #data = np.genfromtxt(fileName, delimiter=',')
        for i in range(num_iter):
            predictions=[]
            test,train=split_data(data,1600)
            for x in range(len(test)):
                neighbors = getNeighborsSAX(train, test[x], data_pars[0],data_pars[1])
                result = getResponse(neighbors)
                predictions.append(result)
                #print('> predicted=' + repr(result) + ', actual=' + repr(test[x][-1]))
            accuracy = getAccuracy(test, predictions)
            print('Accuracy: ' + repr(accuracy) + '%')
            gtruth=[x[-1] for x in test]
            prec = precision_score(gtruth,predictions, average='macro')
            recall= recall_score(gtruth,predictions, average='macro')
            f1=f1_score(gtruth,predictions, average='macro')
            print prec,recall,f1

generated file from 1 data file(s)
sax parameters: (20, 17)
Accuracy: 88.6875%
0.899252516018 0.887243528611 0.885454997827
sax parameters: (20, 20)


KeyboardInterrupt: 

In [352]:
len(predictions)

1458

In [354]:
accuracy = getAccuracy(test[:1458], predictions)
print('Accuracy: ' + repr(accuracy) + '%')
gtruth=[x[-1] for x in test[:1458]]
prec = precision_score(gtruth,predictions, average='macro')
recall= recall_score(gtruth,predictions, average='macro')
f1=f1_score(gtruth,predictions, average='macro')
print prec,recall,f1

Accuracy: 86.89986282578876%
0.875680524346 0.870186629655 0.868231246419


# Serialize some SAX csv files

In [312]:
len(all_sax)

1

In [313]:
len(all_sax[0])

1920

In [314]:
sax=all_sax[0]

In [316]:
s=sax[:100]

In [324]:
f=open('/Users/zangsir/Desktop/sax-rep.csv','a')

for i in sax:
    for j in i[:-1]:
        f.write(j+', ')
    f.write(i[-1]+'\n')
f.close()

# log

# I have moved subsequent experiments to DELL-UBUNTU laptop, so be sure to check that one for updated version.